In [1]:
%cd ..

/home/yuuhanase/workspace/FPTU_FA24_EEG_Artifacts_Recognition


/home/yuuhanase/workspace/FPTU_FA24_EEG_Artifacts_Recognition/thesis/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from FPTU_FA24_EEG_Artifacts_Recognition.config import *
from FPTU_FA24_EEG_Artifacts_Recognition.utils import *

/home/yuuhanase/workspace/FPTU_FA24_EEG_Artifacts_Recognition
2024-11-23 20:42:45,238: INFO: config: TensorFlow version 2.18.0 available.


/home/yuuhanase/workspace/FPTU_FA24_EEG_Artifacts_Recognition/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-23 20:42:45.509792: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 20:42:45.516839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732369365.525178   61902 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:173236

In [3]:
CONFIG_FILE_PATH = Path(r'./config/config.yaml')
PARAMS_FILE_PATH = Path(r'./config/params.yaml')
DATASET_DETAIL_FILE_PATH = Path(r'./config/dataset.yaml')

config_manager = ConfigurationManager(
    CONFIG_FILE_PATH,
    PARAMS_FILE_PATH,
    DATASET_DETAIL_FILE_PATH
)

2024-11-23 20:42:46,151: INFO: common: yaml file 'config/config.yaml' loaded successfully
2024-11-23 20:42:46,153: INFO: common: yaml file 'config/params.yaml' loaded successfully
2024-11-23 20:42:46,154: INFO: common: yaml file 'config/dataset.yaml' loaded successfully


In [4]:
config = config_manager.get_dataset_config()

big_df = {
    'left': [],
    'right': [],
    'both': [],
    'teeth': [],
    'eyebrows': [],
}

for idx, label, position, trial in itter_dataset_file(config):
    filepath = config.filepath_format.format_map(
        {
            'subject_id': idx,
            'label': label,
            'position': position,
            'trial': trial
        }
    )

    raw_roi_filepath = Path(
        os.path.join(config.raw_roi_path, filepath)
    )

    label_df = pd.read_csv(raw_roi_filepath)
    label_df['interval'] = label_df['ROILimits_2'] - label_df['ROILimits_1']
    
    if True in (label_df['interval'] == 0).to_list():
        print(idx, label, position, trial)
    big_df[label].append(label_df)

In [5]:
for label in big_df:
    big_df[label] = pd.concat(big_df[label]).reset_index().drop(columns=['index'])

In [6]:
for label in big_df:
    print(f'############# {label} #############')
    print(big_df[label].describe()['interval']/256)

############# left #############
count    2.648438
mean     0.534425
std      0.132329
min      0.281250
25%      0.445312
50%      0.500000
75%      0.593750
max      1.171875
Name: interval, dtype: float64
############# right #############
count    2.589844
mean     0.494438
std      0.182593
min      0.218750
25%      0.382812
50%      0.449219
75%      0.531250
max      1.449219
Name: interval, dtype: float64
############# both #############
count    2.621094
mean     0.371513
std      0.097086
min      0.128906
25%      0.294922
50%      0.347656
75%      0.445312
max      0.703125
Name: interval, dtype: float64
############# teeth #############
count    2.664062
mean     0.356479
std      0.092866
min      0.171875
25%      0.285156
50%      0.343750
75%      0.416992
max      0.703125
Name: interval, dtype: float64
############# eyebrows #############
count    2.601562
mean     0.489347
std      0.240961
min      0.171875
25%      0.250000
50%      0.496094
75%      0.687500
max

In [9]:
config = config_manager.get_dataset_config()

big_df = {
    'left': [],
    'right': [],
    'both': [],
    'teeth': [],
    'eyebrows': [],
}

for idx, label, position, trial in itter_dataset_file(config):
    filepath = config.filepath_format.format_map(
        {
            'subject_id': idx,
            'label': label,
            'position': position,
            'trial': trial
        }
    )

    raw_data_filepath = Path(
        os.path.join(config.raw_data_path, filepath)
    )

    data_df = pd.read_csv(raw_data_filepath)

    big_df[label].append(len(data_df))

In [10]:
big_df = pd.DataFrame(big_df)
big_df.describe()/256

,left,right,both,teeth,eyebrows
count,0.273438,0.273438,0.273438,0.273438,0.273438
mean,28.422768,26.488114,28.208036,25.987444,26.487946
std,5.593433,4.088354,6.630790,5.403029,4.177586
min,25.125000,20.109375,20.109375,20.109375,20.109375
25%,25.125000,25.125000,25.125000,25.125000,25.125000
50%,25.125000,25.125000,25.125000,25.125000,25.125000
75%,30.140625,25.139648,30.140625,25.125000,25.125000
max,50.203125,40.171875,50.203125,50.250000,40.203125


In [11]:
big_df.sum()/256

left        1989.593750
right       1854.167969
both        1974.562500
teeth       1819.121094
eyebrows    1854.156250
dtype: float64